In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Connecting to the local master cluster 

In [2]:
spark = SparkSession.builder\
        .master("spark://de-zoomcamp.asia-south1-c.c.datatalks-de-409612.internal:7077")\
        .appName("test")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 12:55:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet("data/raw/green/*/*")

In [4]:
df_yellow = spark.read.parquet("data/raw/yellow/*/*")

Renaming Columns 

In [6]:
df_green = df_green.withColumnRenamed("lpep_pickup_datetime","pickup_datetime")\
            . withColumnRenamed("lpep_dropoff_datetime","dropoff_datetime")

In [7]:
df_yellow = df_yellow.withColumnRenamed("tpep_pickup_datetime","pickup_datetime")\
                     .withColumnRenamed("tpep_dropoff_datetime","dropoff_datetime")

Grouping two df's so selecting the common columns and grouping the data 

In [8]:
common_columns = []

yellow_unique_col = set(df_yellow.columns)

for i in df_green.columns:
    if i in yellow_unique_col:
        common_columns.append(i)

Combining two df's

Using F.lit() for the 'word'

In [10]:
df_green = df_green.select(common_columns)\
        .withColumn('service_type',F.lit('green'))

In [11]:
df_yellow = df_yellow.select(common_columns).withColumn("service_type",F.lit('yellow'))

In [12]:
df_comb = df_green.unionAll(df_yellow)

Temp Table

In [13]:
df_comb.registerTempTable('trips_data')

/home/haris/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Using SQL in SPARK

In [14]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc("month", "pickup_datetime") AS revenue_month, 

    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance

FROM 
    trips_data
GROUP by 1, 2, 3
""")

24/02/29 13:12:01 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/02/29 13:12:01 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

df.write.coalase(<Number>).parquet() 
    
coalase means if we have partition then it will reduce the partition to the number of files 

In [56]:
df_result.write.parquet("data/processed/revenue")